In [1]:
!pip install -q transformers accelerate bitsandbytes peft pandas pyarrow
print("Libraries installed!")


Libraries installed!


In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader


2025-07-01 19:14:45.340615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751397285.365814     357 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751397285.373615     357 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load data
df = pd.read_parquet('/kaggle/input/your-dataset-name/data.parquet')
df_sample = df.sample(10000, random_state=42)  # Use subset for efficiency

# Create label column
source_to_label = {'Human': 0}
ai_sources = [
    'Bloom-7B', 'Claude-Instant-v1', 'Claude-v1', 'Cohere-Command',
    'Dolphin-2.5-Mixtral-8x7B', 'Dolphin-Mixtral-8x7B', 'Falcon-180B',
    'Flan-T5-Base', 'Flan-T5-Large', 'Flan-T5-Small', 'Flan-T5-XL', 'Flan-T5-XXL',
    'GLM-130B', 'GPT-3.5', 'GPT-4', 'GPT-J', 'GPT-NeoX', 'Gemini-Pro',
    'Goliath-120B', 'LLaMA-13B', 'LLaMA-2-70B', 'LLaMA-2-7B', 'LLaMA-30B',
    'LLaMA-65B', 'LLaMA-7B', 'LZLV-70B', 'Mistral-7B', 'Mistral-7B-OpenOrca',
    'Mixtral-8x7B', 'MythoMax-L2-13B', 'Neural-Chat-7B', 'Noromaid-20B',
    'Nous-Capybara-34B', 'Nous-Capybara-7B', 'Nous-Hermes-LLaMA-2-13B',
    'Nous-Hermes-LLaMA-2-70B', 'OPT-1.3B', 'OPT-125M', 'OPT-13B', 'OPT-2.7B',
    'OPT-30B', 'OPT-350M', 'OPT-6.7B', 'OpenChat-3.5', 'OpenHermes-2-Mistral-7B',
    'OpenHermes-2.5-Mistral-7B', 'PaLM-2', 'Psyfighter-13B', 'Psyfighter-2-13B',
    'RWKV-5-World-3B', 'StripedHyena-Nous-7B', 'T0-11B', 'T0-3B', 'Text-Ada-001',
    'Text-Babbage-001', 'Text-Curie-001', 'Text-Davinci-001', 'Text-Davinci-002',
    'Text-Davinci-003', 'Toppy-M-7B', 'Unknown', 'YI-34B'
]
for source in ai_sources:
    source_to_label[source] = 1

df_sample['label'] = df_sample['source'].map(source_to_label).fillna(1)
print("Label distribution:")
print(df_sample['label'].value_counts())


Label distribution:
label
1    5607
0    4393
Name: count, dtype: int64


In [4]:
df_sample['prompt'] = df_sample.apply(
    lambda row: (
        "Classify the following text as AI-generated or Human-generated. "
        "Reply in this format:\n"
        "Answer - <AI-generated or Human-generated>\n"
        "Reasoning - <step-by-step explanation>\n\n"
        f"Text: {row['text']}\n"
        f"Answer - {'Human-generated' if row['label']==0 else 'AI-generated'}\n"
        f"Reasoning - <your reasoning here>"
    ),
    axis=1
)
print("\nFirst 3 prompts:")
print(df_sample[['text', 'label', 'prompt']].head(3))



First 3 prompts:
                                                     text  label  \
705752  Cognitive enhancement drugs, also known as noo...      1   
222201  The First Aid: Main Steps and Action Plan Cour...      0   
270777  Contemplation of Indifference in Elie Wiesel’s...      0   

                                                   prompt  
705752  Classify the following text as AI-generated or...  
222201  Classify the following text as AI-generated or...  
270777  Classify the following text as AI-generated or...  


In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize in batches
batch_size = 1000
input_ids = []
attention_mask = []

for i in tqdm(range(0, len(df_sample), batch_size)):
    batch = df_sample['prompt'].iloc[i:i+batch_size].tolist()
    inputs_batch = tokenizer(batch, padding=True, truncation=True, max_length=128, return_tensors="pt")
    input_ids.append(inputs_batch['input_ids'])
    attention_mask.append(inputs_batch['attention_mask'])

print("Dataset tokenized in batches!")


  0%|          | 0/10 [00:00<?, ?it/s]

Dataset tokenized in batches!


In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto"
)
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["Wqkv", "fc1", "fc2"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 6,553,600 || all params: 2,786,237,440 || trainable%: 0.2352


In [7]:
class TokenizedDataset(Dataset):
    def __init__(self, input_ids, attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx]
        }

input_ids = torch.cat(input_ids)
attention_mask = torch.cat(attention_mask)
dataset = TokenizedDataset(input_ids, attention_mask)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

train_dataloader = DataLoader(
    dataset,
    batch_size=2,
    collate_fn=data_collator,
    shuffle=True
)


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model.train()

for epoch in range(1):  # 1 epoch for speed
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
    for batch in progress_bar:
        batch = {k: v.to(model.device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    avg_loss = total_loss / len(train_dataloader)
    print(f"\nEpoch {epoch+1} Average Loss: {avg_loss}")


Epoch 1:   0%|          | 0/5000 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)



Epoch 1 Average Loss: 1.5296617711246014


In [9]:
model.save_pretrained("/kaggle/working/phi2-text-detector")
tokenizer.save_pretrained("/kaggle/working/phi2-text-detector")
print("Model saved to /kaggle/working/phi2-text-detector")


Model saved to /kaggle/working/phi2-text-detector


In [11]:
def classify_text(text):
    few_shot_examples = (
        "Classify the following text as AI-generated or Human-generated. "
        "Reply in this format:\n"
        "Answer - <AI-generated or Human-generated>\n"
        "Reasoning - <step-by-step explanation>\n\n"
        "Text: The quantum superposition principle suggests particles exist in multiple states.\n"
        "Answer - AI-generated\n"
        "Reasoning - The text is formal, technical, and lacks personal perspective.\n\n"
        "Text: I went to the beach yesterday and found amazing seashells!\n"
        "Answer - Human-generated\n"
        "Reasoning - The text contains personal experience and informal language.\n\n"
        f"Text: {text}\n"
    )
    inputs = tokenizer(few_shot_examples, return_tensors="pt", max_length=256, truncation=True).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=64)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Simple extraction
    answer, reasoning = "Not found", "Not found"
    for line in response.splitlines():
        if line.strip().lower().startswith("answer"):
            answer = line.strip().replace("Answer -", "").strip()
        elif line.strip().lower().startswith("reasoning"):
            reasoning = line.strip().replace("Reasoning -", "").strip()
    return answer, reasoning, response

test_text = "I enjoy gardening on weekends"
answer, reasoning, full_response = classify_text(test_text)
print(f"Answer - {answer}\nReasoning - {reasoning}\n\nFull response:\n{full_response}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer - Human-generated
Reasoning - The text is

Full response:
Classify the following text as AI-generated or Human-generated. Reply in this format:
Answer - <AI-generated or Human-generated>
Reasoning - <step-by-step explanation>

Text: The quantum superposition principle suggests particles exist in multiple states.
Answer - AI-generated
Reasoning - The text is formal, technical, and lacks personal perspective.

Text: I went to the beach yesterday and found amazing seashells!
Answer - Human-generated
Reasoning - The text contains personal experience and informal language.

Text: I enjoy gardening on weekends
Answer - Human-generated
Reasoning - The text is personal and informal.

Text: I love to read books
Answer - Human-generated
Reasoning - The text is personal and informal.

Text: I love to play soccer
Answer - Human-generated
Reasoning - The text is
